# Setup

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"heartking","key":"Hmm"}'}

In [3]:
!mkdir -p /root/.config/kaggle
!mv kaggle.json /root/.config/kaggle/
!chmod 600 /root/.config/kaggle/kaggle.json

from kaggle.cli import main
!kaggle competitions download -c digit-recognizer
!unzip digit-recognizer.zip

digit-recognizer.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  digit-recognizer.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def transfer_image(path, tp):
    file = pd.read_csv(path)
    if tp == 'train':
        y = file['label'].to_numpy()
        x = file.drop('label', axis=1).to_numpy()
        x.reshape(len(x), 28, 28)
        return x, y
    else:
        x = file.to_numpy()
        x.reshape(len(x), 28, 28)
        return x

train_x, train_y = transfer_image('train.csv', 'train')
test_x = transfer_image('test.csv', 'test')

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np

class MNISTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx].reshape(28, 28).astype(np.uint8)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = MNISTDataset(train_x, train_y, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

print(f"Training dataset 'trainset' created with {len(trainset)} samples.")
print(f"Shape of the first image after transformation: {trainset[0][0].shape}")
print(f"Label of the first image: {trainset[0][1]}")

Training dataset 'trainset' created with 42000 samples.
Shape of the first image after transformation: torch.Size([1, 28, 28])
Label of the first image: 1


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

epoch = 20
for epoch in range(epoch):
    running_loss = 0.0
    for images, label in tqdm.tqdm(trainloader):
        images, label = images.to(device), label.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, label)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch #{epoch + 1}:', running_loss)

100%|██████████| 657/657 [00:06<00:00, 97.16it/s] 


Epoch #1: 221.7940027974546


100%|██████████| 657/657 [00:05<00:00, 113.50it/s]


Epoch #2: 55.249945333227515


100%|██████████| 657/657 [00:06<00:00, 100.81it/s]


...

Epoch #18: 4.734691533900332


100%|██████████| 657/657 [00:08<00:00, 75.63it/s]


Epoch #19: 5.913042274772579


100%|██████████| 657/657 [00:06<00:00, 102.83it/s]

Epoch #20: 5.2065606925255


In [8]:
# test.csv

testset = MNISTDataset(test_x, np.zeros(len(test_x)), transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

model.eval()
predictions = []

with torch.no_grad():
    for images, _ in tqdm.tqdm(testloader):
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())

submission = pd.DataFrame({
    "ImageId": range(1, len(predictions) + 1),
    "Label": predictions
})

submission.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created successfully.")
print(submission.head())

100%|██████████| 438/438 [00:04<00:00, 106.48it/s]


Submission file 'submission.csv' created successfully.
   ImageId  Label
0        1      2
1        2      0
2        3      9
3        4      9
4        5      3


In [9]:
!kaggle competitions submit -c digit-recognizer -f submission.csv -m "Message"

100% 208k/208k [00:00<00:00, 344kB/s]
Successfully submitted to Digit Recognizer